In [1]:
%%writefile utility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re
import csv


################
# File Reading #
################
def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)

def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting utility.py


In [2]:
%%writefile readfile.yaml
file_type: csv
dataset_name: testfile
file_name: reviews
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
skip_leading_columns: 1
columns: 
    - user
    - rating
    - comment
    - ID
    - name

Overwriting readfile.yaml


In [3]:
# Read config file
import utility as util
config_data = util.read_config_file("readfile.yaml")

In [4]:
config_data['inbound_delimiter']

','

In [5]:
import pandas as pd
# read the file using config file
file_type = config_data['file_type']
source_file = "./" + config_data['file_name'] + f'.{file_type}'
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df = df.to_csv(source_file,config_data['outbound_delimiter'])
df = pd.read_csv(source_file,config_data['outbound_delimiter'])
#print("",source_file)

In [6]:
df.head()

,Unnamed: 0,Unnamed: 0.1,user,rating,comment,ID,name
0,0,0,Torsten,10.0,NaN,30549,Pandemic
1,1,1,mitnachtKAUBO-I,10.0,Hands down my favorite new game of BGG CON 200...,30549,Pandemic
2,2,2,avlawn,10.0,I tend to either love or easily tire of co-op ...,30549,Pandemic
3,3,3,Mike Mayer,10.0,NaN,30549,Pandemic
4,4,4,Mease19,10.0,This is an amazing co-op game. I play mostly ...,30549,Pandemic


In [7]:
print("columns of files are:" ,df.columns)
print("number of columns is:" ,df.shape[1])
print("number of rows is:" ,df.shape[0])

columns of files are: Index(['Unnamed: 0', 'Unnamed: 0.1', 'user', 'rating', 'comment', 'ID',
       'name'],
      dtype='object')
number of columns is: 7
number of rows is: 18964807


In [8]:
import os
print("the size is:" ,os.stat("reviews.csv"))

the size is: os.stat_result(st_mode=33206, st_ino=24206847997136165, st_dev=1176961840, st_nlink=1, st_uid=0, st_gid=0, st_size=1781209948, st_atime=1660109559, st_mtime=1660109559, st_ctime=1660107883)
